# Notebook Imports

In [33]:
import pandas as pd
import numpy as np

# Constants

In [34]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train_data.txt'
TESTING_DATA_FILE = 'SpamData/02_Training/test_data.txt'


TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

## Read and load features from .txt file into a numpy array

In [35]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype='int')

In [36]:
sparse_test_data = np.loadtxt(TESTING_DATA_FILE, delimiter=' ', dtype='int')

In [37]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [38]:
sparse_train_data[-5:]

array([[5795, 2052,    0,    1],
       [5795, 2059,    0,    2],
       [5795, 2295,    0,    1],
       [5795, 2363,    0,    1],
       [5795, 2418,    0,    1]])

In [39]:
print('nr of rows in training dataset: ', sparse_train_data.shape[0])
print('nr of rows in testing dataset: ', sparse_test_data.shape[0])

nr of rows in training dataset:  258358
nr of rows in testing dataset:  117611


In [40]:
print('nr of emails in training dataset: ', np.unique(sparse_train_data[:,0]).size)
print('nr of emails in test dataset: ', np.unique(sparse_test_data[:,0]).size)

nr of emails in training dataset:  4015
nr of emails in test dataset:  1724


## How to create an empty dataframe

In [41]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[0:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [42]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [43]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace = True)
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create a full matrix from a sparse matrix

In [44]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    From a full matrix form a sparse matrix. Return a pandas dataframe.
    Keyword arguements:
    sparse matrix -- numpy array.
    nr_words -- size of the vocabulary. total nr. of tokens.
    doc_idx -- position of the document id in the sparse matrix. default:1st column.
    word_idx -- position of the word id in the sparse matrix. default:2nd column.
    cat_idx -- position of the label( spam is 1, nonspam is 0). default:3rd column.
    freq_idx -- position of occurence of word in the sparse matrix. default:4th column.
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace = True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace = True)
    return full_matrix
    

In [45]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 14.2 s


In [46]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,13,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the probability of spam

In [47]:
full_train_data.CATEGORY.size

4015

In [48]:
full_train_data.CATEGORY.sum()

1250

In [49]:
prob_sum = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is: ', prob_sum)

Probability of spam is:  0.31133250311332505


In [50]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

Probability of spam is 0.31133250311332505


## Total Nr of words/ tokens

In [51]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
email_lengths = full_train_features.sum(axis=1)
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    181
4     43
dtype: int64

In [53]:
total_wc = email_lengths.sum()
total_wc

431001

## Number of Tokens in spam and ham emails

In [54]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape[0]

1250

In [55]:
spam_wc = spam_lengths.sum()
spam_wc

178628

In [56]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape[0]

2765

In [57]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

252373

In [58]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [59]:
print('Nr of words per spam emails {:.0f}' .format(spam_wc / spam_lengths.shape[0]))
print('Nr of words per ham emails {:.2f}' .format(nonspam_wc / ham_lengths.shape[0]))

Nr of words per spam emails 143
Nr of words per ham emails 91.27


## Summing up the tokens occuring in spam

In [60]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.shape[0]

1250

In [61]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape[0]

2500

In [62]:
summed_spam_tokens.tail()

2495     3
2496    27
2497     7
2498    19
2499     1
dtype: int64

## Summing up the tokens occuring in ham

In [63]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.shape[0]

2765

In [64]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens.shape[0]

2500

In [65]:
summed_ham_tokens.tail()

2495    33
2496    15
2497    30
2498    16
2499    38
dtype: int64

## P(token | spam) - Probability that the token occurs given that the email is spam

In [66]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]
prob_tokens_spam.sum()

1.0

## P(token | ham) - Probability that the token occurs given that the email is nonspam

In [67]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam[:5]
prob_tokens_nonspam.sum()

1.0

## P(Token) - Probability that token occurs

In [68]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all[:5]
prob_tokens_all.sum()

1.0

# Saving the trained model

In [69]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test data

In [70]:
%%time
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 6.83 s


In [71]:
full_test_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,0,0,1,4,2,1,2,4,1,2,...,0,0,0,0,0,0,0,0,0,0
12,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,2,1,1,2,0,0,3,0,4,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
y_test = full_test_data.CATEGORY
y_test

DOC_ID
8       1
12      1
14      1
15      1
17      1
       ..
5783    0
5786    0
5788    0
5792    0
5793    0
Name: CATEGORY, Length: 1724, dtype: int64

## Saving Test data 

In [74]:
np.savetxt(TEST_FEATURE_MATRIX, X_test)
np.savetxt(TEST_TARGET_FILE, y_test)